In [ ]:
f_func <- "./functions.ipynb"
eval(parse(text = system2('jupyter', c('nbconvert', f_func, '--to=script', '--stdout'), stdout = TRUE)))

f_data_loading <- "./data_loading.ipynb"
eval(parse(text = system2('jupyter', c('nbconvert', f_data_loading, '--to=script', '--stdout'), stdout = TRUE)))

In [ ]:
directory_path <- "../../output/cfspi/samples/results/stats/"
files <- list.files(directory_path, full.names = TRUE)
filtered_files <- files[grep("_R1_01_", files)]

results <- data.frame(sample_id = character(), 
                      nr_QC_reads = numeric(), 
                      stringsAsFactors = FALSE)

for (file in filtered_files) {
    base_name <- sub("^(.*)_R1_01_.*$", "\\1", basename(file))
    value <- readLines(file, n = 1)

    results <- results %>% add_row(sample_id = base_name, 
                                   nr_QC_reads = as.numeric(value))
}

suppl_table <- results %>% 
    filter(sample_id %in% samples_metadata$sample_id) %>% 
    filter(sample_id %nin% samples_metadata$sample_id[
        samples_metadata$origin == "controls"]) %>% 
    filter(sample_id %nin% samples_metadata$sample_id[
        samples_metadata$type == "ds-wcDNA"])

write.csv(suppl_table, "../../output/PP_figures/suppl_table.csv", 
          row.names = FALSE)

In [ ]:
samples_metadata %>% 
    filter(origin != "controls") %>% 
    filter(sample_type != "ds-wcDNA BAL") %>% 
    select(patient) %>% unlist() %>% 
    as.character() %>% unique() %>% length()

samples_metadata %>% 
    filter(sample_type == "ssLP plasma") %>% filter(origin == "IPA samples")

samples_metadata %>% 
    filter(origin != "controls") %>% 
    filter(sample_type != "ds-wcDNA BAL") %>%  
    group_by(sample_type) %>% 
    summarise(nr_samples = n())

samples_metadata %>% 
    filter(origin != "controls") %>% 
    filter(sample_type != "ds-wcDNA BAL") %>%  
    group_by(origin) %>% 
    summarise(nr_samples = n())

samples_metadata %>% 
    filter(origin != "controls") %>% 
    filter(sample_type != "ds-wcDNA BAL") %>%  
    group_by(sample_type, origin) %>% 
    summarise(nr_samples = n())

In [ ]:
host_nonMT$sample_id %>% unique() %>% length()

In [ ]:
#SOME NUMBERS FOR THE MANUSCRIPT
unique_sample_id_count <- host_nonMT %>% 
    mutate(sample_short = substr(sample_id, 1,3)) %>% 
    ungroup() %>% 
    summarise(unique_count = n_distinct(sample_short), .groups = "keep")
unique_sample_id_count

numbers_per_sample_type <- host_nonMT %>%
    group_by(sample) %>%
    summarise(unique_count = n_distinct(patient), .groups = "keep")
numbers_per_sample_type

host_nonMT %>% filter(sample == "plasma") %>%
    group_by(origin) %>% 
    summarise(unique_count = n_distinct(patient), .groups = "keep")

host_nonMT %>% filter(sample == "BAL") %>%
    group_by(origin) %>% 
    summarise(unique_count = n_distinct(patient), .groups = "keep")

average_fragment_length <- host_nonMT %>% 
    group_by(TLEN, sample_type, type) %>% summarise(mnsCount = mean(nsCount, na.rm = TRUE), .groups = "keep") %>% 
    group_by(sample_type, type) %>% 
    summarise(avg_TLEN = round(sum(TLEN * mnsCount, na.rm = TRUE) / sum(mnsCount, na.rm = TRUE), digits = 0), 
              .groups = "keep")
average_fragment_length

In [ ]:
get_index_for_valley(host_nonMT_summarized, sample_type_tmp = "ssLP plasma", tlen_min = 170)[[1]] %>% 
    filter(TLEN >160) %>% filter(type == "peak")
get_index_for_valley(host_nonMT_summarized, sample_type_tmp = "dsLP plasma", tlen_min = 170)[[1]] %>% 
    filter(TLEN >160) %>% filter(type == "peak")
get_index_for_valley(host_nonMT_summarized, sample_type_tmp = "ssLP BAL", tlen_min = 170)[[1]] %>% 
    filter(TLEN >160) %>% filter(type == "peak")
get_index_for_valley(host_nonMT_summarized, sample_type_tmp = "dsLP BAL", tlen_min = 170)[[1]] %>% 
    filter(TLEN >160) %>% filter(type == "peak")

get_index_for_valley(host_nonMT_summarized, sample_type_tmp = "ssLP plasma")[[2]]
get_index_for_valley(host_nonMT_summarized, sample_type_tmp = "dsLP plasma")[[2]]
get_index_for_valley(host_nonMT_summarized, sample_type_tmp = "ssLP BAL")[[2]]
get_index_for_valley(host_nonMT_summarized, sample_type_tmp = "dsLP BAL")[[2]]

In [ ]:
rbind(plasma_sizeSelection_meta, BAL_sizeSelection_meta) %>%
    filter(origin != "control", 
           taxName == "Aspergillus fumigatus") %>%
    select(sample_id, new_Filter, QC_number_reads, sample_type, origin, type) %>% 
    filter(new_Filter %in% c("noFilter", "35-1000")) %>%     
    pivot_wider(names_from = new_Filter, values_from = QC_number_reads, names_prefix = "QC#_") %>% 
    mutate(perc_merged = `QC#_35-1000`/`QC#_noFilter`) %>% 
    group_by(sample_type) %>% 
    summarize(mean_perc_merged = mean(perc_merged))